In [1]:
import tensorflow

In [2]:
import pandas as pd
def filesave(dataframe,name):
    dataframe.to_csv(name,sep="\t", encoding="utf-8",index=False)
def fileread(name):
    return pd.read_csv(name,sep="\t",encoding="utf-8")

In [5]:
test=fileread('./testdata/chosun_test.csv')

In [7]:
from tensorflow.keras.models import load_model
from konlpy.tag import Okt
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
max_len = 42


with open("senti_tokenizer.pickle","rb") as fr:
    tokenizer = pickle.load(fr)

loaded_model = load_model('LSTMGRU.h5')

okt=Okt()

In [9]:

def predict_text(text):
    new_text = okt.morphs(text, stem=True)
    new_text = [word for word in new_text if not word in stopwords]
    encoded = tokenizer.texts_to_sequences([new_text])
    pad_encoded_text = pad_sequences(encoded,maxlen=max_len)
    score = float(loaded_model.predict(pad_encoded_text))
    return score

def predict_senti(text):
    idx2label={0:"부정",1:"긍정"}
    score_list=[]
    for i in text:
        score_list.append(idx2label[round(predict_text(i))])
    return score_list

def predict_senti2(text):
    idx2label={0:"부정",1:"긍정",-1:"중립"}
    score_list = []
    for i in text:
        score = predict_text(i)
        if score >= 0.7:
            label = 1
        elif score <= 0.3:
            label = 0
        else:
            label = -1
        score_list.append(idx2label[label])
    return score_list



In [16]:
cnt=0
for i in range(len(test)):
    print((test.iloc[i])['title'])
    print("----")
    cnt+=1
    if cnt>10:
        break

신한금융, 3분기 순익 1.6조… 역대 최대 실적
----
자금시장 경색에… 한은 ‘베이비스텝’ 전망 나와
----
일본은행 두번째 복면개입?  엔·달러 환율 149엔대에서 갑자기 4엔 하락
----
매달 10만원씩 6년 부었는데 이자 12만원? 낮아도 너무 낮은 금리에 ‘저축보험’ 끊는다
----
작년 종부세 수정 요구 1481건, 1년새 80% 폭증… 절반이 받아들여졌다
----
[7NEWS] 질주하던 테슬라, 中에 발목 잡히나... 차량 가격 최대 9% 인하
----
시진핑 3기 출범하자마자... 홍콩 증시 6.36% 폭락
----
가스공사 초우량 채권도 유찰…'채권안정 50조’ 대책에도 시장 냉담
----
‘일하는 노년’… 60세 이상 근로소득 4년새 84% 늘었다
----
350개 공공기관, 내년까지 정원 6700여명 줄이나
----


In [17]:
copied_df = test.copy()
copied_df["title_senti"]=None

In [18]:
copied_df

,title,content,date,section,url,title_senti
0,"신한금융, 3분기 순익 1.6조… 역대 최대 실적",조용병 신한금융지주 회장이 고객 중심 경영을 강조하고 있다.신한금융이 올해 3분기(...,2022.10.25 13:52,경제,https://www.chosun.com/economy/economy_general...,None
1,자금시장 경색에… 한은 ‘베이비스텝’ 전망 나와,‘레고랜드 사태’가 뇌관이 된 자금 시장 경색이 쉽게 풀릴 기미를 보이지 않으면서 ...,2022.10.25 04:16,경제,https://www.chosun.com/economy/economy_general...,None
2,일본은행 두번째 복면개입? 엔·달러 환율 149엔대에서 갑자기 4엔 하락,지난 20일 스즈키 슌이치 일본 재무상이 취재진과 만나는 모습. /로이터 연합뉴스달...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...,None
3,매달 10만원씩 6년 부었는데 이자 12만원? 낮아도 너무 낮은 금리에 ‘저축보험’...,은행 수신금리가 나날이 오르자 오래 전 저축보험을 가입했다가 중도해지에 나서는 사람...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...,None
4,"작년 종부세 수정 요구 1481건, 1년새 80% 폭증… 절반이 받아들여졌다",지난해 납세자들이 종합부동산세(종부세)를 과도하게 냈다며 수정을 요구한 경정 청구가...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...,None
5,"[7NEWS] 질주하던 테슬라, 中에 발목 잡히나... 차량 가격 최대 9% 인하",7NEWS 뉴스레터 구독하기 ☞https://page.stibee.com/subsc...,2022.10.24 19:00,경제,https://www.chosun.com/economy/economy_general...,None
6,시진핑 3기 출범하자마자... 홍콩 증시 6.36% 폭락,‘1인 지배 체계’를 굳힌 시진핑 정권 3기가 대외 강경 노선으로 일관할 것이라는 ...,2022.10.24 18:44,경제,https://www.chosun.com/economy/economy_general...,None
7,가스공사 초우량 채권도 유찰…'채권안정 50조’ 대책에도 시장 냉담,추경호 경제부총리 겸 기획재정부 장관(가운데)을 비롯한 경제부처장들이 23일 오후 ...,2022.10.24 16:57,경제,https://www.chosun.com/economy/economy_general...,None
8,‘일하는 노년’… 60세 이상 근로소득 4년새 84% 늘었다,60세 이상 고령층의 근로소득이 지난 4년(2016~2020년)간 84%나 상승한 ...,2022.10.24 16:42,경제,https://www.chosun.com/economy/economy_general...,None
9,"350개 공공기관, 내년까지 정원 6700여명 줄이나",공공기관이 내년까지 정원을 6700명 넘게 줄이는 방침을 세운 것으로 나타났다.24...,2022.10.24 16:27,경제,https://www.chosun.com/economy/economy_general...,None


In [ ]:
copied_df = fileread("chosun_senti_df.csv")

In [35]:
def save_senti_df(count):
    for i in range(count,count+100):
        if i >= len(copied_df):
            break
        sentence = (copied_df.iloc[i])['title']
        (copied_df.iloc[i])['title_senti']= predict_text(sentence)
    filesave(copied_df,"chosun_senti_df.csv")
    print(count+100,"까지 감정 분석 후 저장")

In [36]:
count=0
for i in range((len(copied_df)//100) +1):
    save_senti_df(count)
    count+=10

1/1 [==============================] - 0s 133ms/step
100 까지 감정 분석 후 저장


In [37]:
chosun_senti_df = fileread("chosun_senti_df.csv")

In [38]:
chosun_senti_df

,title,content,date,section,url,title_senti
0,"신한금융, 3분기 순익 1.6조… 역대 최대 실적",조용병 신한금융지주 회장이 고객 중심 경영을 강조하고 있다.신한금융이 올해 3분기(...,2022.10.25 13:52,경제,https://www.chosun.com/economy/economy_general...,0.934428
1,자금시장 경색에… 한은 ‘베이비스텝’ 전망 나와,‘레고랜드 사태’가 뇌관이 된 자금 시장 경색이 쉽게 풀릴 기미를 보이지 않으면서 ...,2022.10.25 04:16,경제,https://www.chosun.com/economy/economy_general...,0.269128
2,일본은행 두번째 복면개입? 엔·달러 환율 149엔대에서 갑자기 4엔 하락,지난 20일 스즈키 슌이치 일본 재무상이 취재진과 만나는 모습. /로이터 연합뉴스달...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...,0.004256
3,매달 10만원씩 6년 부었는데 이자 12만원? 낮아도 너무 낮은 금리에 ‘저축보험’...,은행 수신금리가 나날이 오르자 오래 전 저축보험을 가입했다가 중도해지에 나서는 사람...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...,0.952862
4,"작년 종부세 수정 요구 1481건, 1년새 80% 폭증… 절반이 받아들여졌다",지난해 납세자들이 종합부동산세(종부세)를 과도하게 냈다며 수정을 요구한 경정 청구가...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...,0.012476
5,"[7NEWS] 질주하던 테슬라, 中에 발목 잡히나... 차량 가격 최대 9% 인하",7NEWS 뉴스레터 구독하기 ☞https://page.stibee.com/subsc...,2022.10.24 19:00,경제,https://www.chosun.com/economy/economy_general...,0.064300
6,시진핑 3기 출범하자마자... 홍콩 증시 6.36% 폭락,‘1인 지배 체계’를 굳힌 시진핑 정권 3기가 대외 강경 노선으로 일관할 것이라는 ...,2022.10.24 18:44,경제,https://www.chosun.com/economy/economy_general...,0.035841
7,가스공사 초우량 채권도 유찰…'채권안정 50조’ 대책에도 시장 냉담,추경호 경제부총리 겸 기획재정부 장관(가운데)을 비롯한 경제부처장들이 23일 오후 ...,2022.10.24 16:57,경제,https://www.chosun.com/economy/economy_general...,0.335938
8,‘일하는 노년’… 60세 이상 근로소득 4년새 84% 늘었다,60세 이상 고령층의 근로소득이 지난 4년(2016~2020년)간 84%나 상승한 ...,2022.10.24 16:42,경제,https://www.chosun.com/economy/economy_general...,0.267937
9,"350개 공공기관, 내년까지 정원 6700여명 줄이나",공공기관이 내년까지 정원을 6700명 넘게 줄이는 방침을 세운 것으로 나타났다.24...,2022.10.24 16:27,경제,https://www.chosun.com/economy/economy_general...,0.152099


In [84]:
chosun_senti_content_df = chosun_senti_df.copy()

In [85]:
c=chosun_senti_content_df.to_dict('records')

In [86]:
c[0]

{'title': '신한금융, 3분기 순익 1.6조… 역대 최대 실적',
 'content': '조용병 신한금융지주 회장이 고객 중심 경영을 강조하고 있다.신한금융이 올해 3분기(7~9월) 1조5946억원의 순이익을 달성했다고 25일 밝혔다. 이는 전분기 대비 20.8% 증가한 것으로 분기 기준 역대 최대 실적이다.누적 기준으로도 역대급 실적이다. 신한금융그룹은 지난 1~3분기 누적으로 4조3154억원의 순이익을 올렸다. 전년 동기 대비 21.2% 증가한 수치로 사상 최대 규모다. 금리 상승기, 이자 이익이 증가한데다 서울 여의도 신한투자증권(구 신한금융투자) 사옥 매각이익(4438억원)으로 자본 효율화를 해둔 결과로 풀이된다.3분기 비이자이익(6092억원)이 전분기 대비 28.8% 감소했지만, 이자이익이 2조7160억원으로 크게 증가한 덕분에 타격이 적었다. 핵심 계열사인 신한은행의 3분기 순이자마진(NIM)은 1.68%로 전분기(1.63%) 보다 훨씬 개선됐다. 기업대출 또한 전년 말 대비 8.6% 증가하는 등 대출 자산 성장도 계속되는 모습을 보였다.3분기 신한금융의 영업외이익은 4996억원, 누적 영업외이익은 5881억원으로 나타났다.신한금융의 3분기 대손비용은 2506억원으로 전 분기 대비 30% 줄었다. 2분기에 추가 충당금 2245억원을 쌓아두는 등 선제적으로 손실 흡수 능력을 갖춰놓은 데 따른 기저효과다. 신한금융의 9월말 그룹 BIS자기자본비율은 15.9%, 보통주자본비율(CET1비율)은 12.7%로 규제 비율을 충족했다.한편 신한금융지주는 지난 6일 보통주 배당금 400원 지급, 1500억원 규모의 자사주 매입과 소각을 결의했다. 신한금융 관계자는 “지난해 2분기 이후 주주환원 정책의 가시성을 확보하기 위해 분기 배당을 정례화했으며, 이번 배당도 이러한 정책의 연장선에서 결정됐다”고 설명했다.하나금융그룹도 호실적을 거뒀다. 하나금융그룹은 올해 3분기, 전년 동기 대비 20.8% 증가한 1조1219억원의 순익을 냈다고 밝혔다. 1~3분기 누적으

In [87]:
counter_test=Counter(["긍정","부정","중립","긍정"])

In [88]:
z = {**c[0], **dict(counter_test)}

In [89]:
z

{'title': '신한금융, 3분기 순익 1.6조… 역대 최대 실적',
 'content': '조용병 신한금융지주 회장이 고객 중심 경영을 강조하고 있다.신한금융이 올해 3분기(7~9월) 1조5946억원의 순이익을 달성했다고 25일 밝혔다. 이는 전분기 대비 20.8% 증가한 것으로 분기 기준 역대 최대 실적이다.누적 기준으로도 역대급 실적이다. 신한금융그룹은 지난 1~3분기 누적으로 4조3154억원의 순이익을 올렸다. 전년 동기 대비 21.2% 증가한 수치로 사상 최대 규모다. 금리 상승기, 이자 이익이 증가한데다 서울 여의도 신한투자증권(구 신한금융투자) 사옥 매각이익(4438억원)으로 자본 효율화를 해둔 결과로 풀이된다.3분기 비이자이익(6092억원)이 전분기 대비 28.8% 감소했지만, 이자이익이 2조7160억원으로 크게 증가한 덕분에 타격이 적었다. 핵심 계열사인 신한은행의 3분기 순이자마진(NIM)은 1.68%로 전분기(1.63%) 보다 훨씬 개선됐다. 기업대출 또한 전년 말 대비 8.6% 증가하는 등 대출 자산 성장도 계속되는 모습을 보였다.3분기 신한금융의 영업외이익은 4996억원, 누적 영업외이익은 5881억원으로 나타났다.신한금융의 3분기 대손비용은 2506억원으로 전 분기 대비 30% 줄었다. 2분기에 추가 충당금 2245억원을 쌓아두는 등 선제적으로 손실 흡수 능력을 갖춰놓은 데 따른 기저효과다. 신한금융의 9월말 그룹 BIS자기자본비율은 15.9%, 보통주자본비율(CET1비율)은 12.7%로 규제 비율을 충족했다.한편 신한금융지주는 지난 6일 보통주 배당금 400원 지급, 1500억원 규모의 자사주 매입과 소각을 결의했다. 신한금융 관계자는 “지난해 2분기 이후 주주환원 정책의 가시성을 확보하기 위해 분기 배당을 정례화했으며, 이번 배당도 이러한 정책의 연장선에서 결정됐다”고 설명했다.하나금융그룹도 호실적을 거뒀다. 하나금융그룹은 올해 3분기, 전년 동기 대비 20.8% 증가한 1조1219억원의 순익을 냈다고 밝혔다. 1~3분기 누적으

In [90]:
test_df = pd.DataFrame(columns=chosun_senti_content_df.columns)
test_df.append(z,ignore_index=True)

,title,content,date,section,url,title_senti,긍정,부정,중립
0,"신한금융, 3분기 순익 1.6조… 역대 최대 실적",조용병 신한금융지주 회장이 고객 중심 경영을 강조하고 있다.신한금융이 올해 3분기(...,2022.10.25 13:52,경제,https://www.chosun.com/economy/economy_general...,0.934428,2.0,1.0,1.0


In [111]:
temp_df = pd.DataFrame(columns=chosun_senti_content_df.columns)

In [113]:
filesave(temp_df,"chosun_senti_content_df.csv")

In [114]:

from collections import Counter
def save_senti_content_df(count,df):
    for i in range(count,count+100):
        if i>=len(c):
            break
        contents = (c[i])['content']
        sentences = str(contents).replace(". ","\n").split("\n")
        counter = Counter(predict_senti2(sentences))

        tempdict=dict(counter)
        merge_dict = {**c[i],**tempdict}
        df = df.append(merge_dict, ignore_index=True)
    filesave(df,"chosun_senti_content_df.csv")
    print(count+100,"까지 저장")

    

In [115]:
count=0
for _ in range((len(c)//100)+1):
    temp_df = fileread("chosun_senti_content_df.csv")
    save_senti_content_df(count,temp_df)
    count+=100

1/1 [==============================] - 0s 71ms/step
100 까지 저장


In [118]:
temp_df = fileread("chosun_senti_content_df.csv")

In [119]:
temp_df

,title,content,date,section,url,title_senti,긍정,부정,중립
0,"신한금융, 3분기 순익 1.6조… 역대 최대 실적",조용병 신한금융지주 회장이 고객 중심 경영을 강조하고 있다.신한금융이 올해 3분기(...,2022.10.25 13:52,경제,https://www.chosun.com/economy/economy_general...,0.934428,2.0,3.0,9.0
1,자금시장 경색에… 한은 ‘베이비스텝’ 전망 나와,‘레고랜드 사태’가 뇌관이 된 자금 시장 경색이 쉽게 풀릴 기미를 보이지 않으면서 ...,2022.10.25 04:16,경제,https://www.chosun.com/economy/economy_general...,0.269128,NaN,4.0,2.0
2,일본은행 두번째 복면개입? 엔·달러 환율 149엔대에서 갑자기 4엔 하락,지난 20일 스즈키 슌이치 일본 재무상이 취재진과 만나는 모습. /로이터 연합뉴스달...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...,0.004256,3.0,8.0,1.0
3,매달 10만원씩 6년 부었는데 이자 12만원? 낮아도 너무 낮은 금리에 ‘저축보험’...,은행 수신금리가 나날이 오르자 오래 전 저축보험을 가입했다가 중도해지에 나서는 사람...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...,0.952862,5.0,6.0,6.0
4,"작년 종부세 수정 요구 1481건, 1년새 80% 폭증… 절반이 받아들여졌다",지난해 납세자들이 종합부동산세(종부세)를 과도하게 냈다며 수정을 요구한 경정 청구가...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...,0.012476,1.0,6.0,4.0
5,"[7NEWS] 질주하던 테슬라, 中에 발목 잡히나... 차량 가격 최대 9% 인하",7NEWS 뉴스레터 구독하기 ☞https://page.stibee.com/subsc...,2022.10.24 19:00,경제,https://www.chosun.com/economy/economy_general...,0.064300,NaN,24.0,8.0
6,시진핑 3기 출범하자마자... 홍콩 증시 6.36% 폭락,‘1인 지배 체계’를 굳힌 시진핑 정권 3기가 대외 강경 노선으로 일관할 것이라는 ...,2022.10.24 18:44,경제,https://www.chosun.com/economy/economy_general...,0.035841,4.0,15.0,2.0
7,가스공사 초우량 채권도 유찰…'채권안정 50조’ 대책에도 시장 냉담,추경호 경제부총리 겸 기획재정부 장관(가운데)을 비롯한 경제부처장들이 23일 오후 ...,2022.10.24 16:57,경제,https://www.chosun.com/economy/economy_general...,0.335938,3.0,17.0,3.0
8,‘일하는 노년’… 60세 이상 근로소득 4년새 84% 늘었다,60세 이상 고령층의 근로소득이 지난 4년(2016~2020년)간 84%나 상승한 ...,2022.10.24 16:42,경제,https://www.chosun.com/economy/economy_general...,0.267937,NaN,5.0,2.0
9,"350개 공공기관, 내년까지 정원 6700여명 줄이나",공공기관이 내년까지 정원을 6700명 넘게 줄이는 방침을 세운 것으로 나타났다.24...,2022.10.24 16:27,경제,https://www.chosun.com/economy/economy_general...,0.152099,2.0,3.0,2.0
